In [1]:
import duckdb 
con = duckdb.connect()
import pandas as pd
import os
import requests
from io import StringIO
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())
AWS_ACCESS_KEY = os.getenv("AWS_ACCESS_KEY_MINIO")
AWS_SECRET_KEY = os.getenv("AWS_SECRET_KEY_MINIO")  # Ajuste conforme necessário para capturar o secret key
HOST_MINIO = os.getenv("HOST_MINIO")

In [2]:
con.execute(f"""
    CREATE SECRET my_minio_secret (
        TYPE 'S3',
        KEY_ID '{AWS_ACCESS_KEY}',
        SECRET '{AWS_SECRET_KEY}',
        REGION 'us-east-1',
        ENDPOINT '{HOST_MINIO}:9000',
        URL_STYLE 'path',
        USE_SSL false
    );
""")


In [3]:
path_csv = 'C:\\COMEX'
path_minio = 's3://landing/comex'

#### Testar conexão

In [ ]:
con.sql(f""" 
           SELECT * FROM '{path_csv}\\IMP_EXP\\EXP_COMPLETA.csv' limit 5;
        """)


In [ ]:
con.sql(f""" 
           SELECT * FROM '{path_csv}\\IMP_EXP\\EXP_COMPLETA.csv' limit 5;
        """)

#### Subir manualmente os arquivos


In [119]:
con.sql(f""" 
           COPY (SELECT * FROM '{path_csv}\\IMP_EXP\\EXP_COMPLETA.csv') TO '{path_minio}/EXP-COMPLETA.parquet';
        """)

#### Subir manualmente os arquivos

In [120]:

con.sql(f""" 
           COPY (SELECT * FROM '{path_csv}\\IMP_EXP\\IMP_COMPLETA.csv') TO '{path_minio}/IMP-COMPLETA.parquet';
        """)

#### Conferir os anos

In [ ]:
con.sql(f"""
    SELECT distinct CO_ANO 
    FROM read_parquet('{path_minio}/IMP/*.parquet')
""")

#### Subir automaticamente EXP e IMP

In [ ]:
lista_exp_imp = ['EXP', 'IMP'] 
lista_anos = [2023, 2024] 

for tabela in lista_exp_imp:
    for ano in lista_anos:
        con.sql(f""" 
                COPY (SELECT * FROM read_csv('https://balanca.economia.gov.br/balanca/bd/comexstat-bd/ncm/{tabela}_{ano}.csv', ignore_errors=true)) 
                TO '{path_minio}/{tabela}/{ano}.parquet' (FORMAT 'parquet');
            """)

#### Loop para acessar todas as combinações de tabelas e anos sem sobrescrever

In [ ]:
# Função para baixar, converter e processar os CSVs
def process_csvs_to_parquet(lista, path_minio):
    for tabela in lista:
        # URL do arquivo CSV
        url = f"https://balanca.economia.gov.br/balanca/bd/tabelas/{tabela}.csv"
        response = requests.get(url, verify=False)
        
        csv_data = StringIO(response.text)
        df = pd.read_csv(csv_data, sep=';', encoding='ISO-8859-1')

        # Salvar diretamente como Parquet no caminho especificado
        con.sql(f""" 
            COPY (SELECT * FROM df) 
            TO '{path_minio}/{tabela}.parquet' (FORMAT 'parquet');
         """)
        

# Defina a lista de tabelas
lista = ['PAIS_BLOCO', 'UF', 'PAIS', 'UF_MUN', 'NCM_SH', 'NCM_FAT_AGREG','NCM']

# Caminho de destino (path_minio) para salvar os arquivos parquet
path_minio = 's3://landing/comex' # Altere para o seu caminho

# Executar a função
process_csvs_to_parquet(lista, path_minio)

In [123]:
con.close()